# Bibliotecas

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display
import toml
import gspread
from google.oauth2.service_account import Credentials
from auxiliar.arrumar_bases import *
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import random

def get_sheetdata(sheet_name):
    # Load credentials from the .streamlit/secrets.toml file
    secrets = toml.load(".streamlit/secrets.toml")
    creds_info = secrets["connections"]["gsheets"]

    # Define scopes for accessing Google Sheets and Drive
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    # Create credentials and authorize the gspread client
    creds = Credentials.from_service_account_info(creds_info, scopes=scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet using the URL from the toml file
    spreadsheet = client.open_by_url(creds_info["spreadsheet"])

    # Select the first worksheet and fetch all records
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()

    df = pd.DataFrame(data)

    return df

def update_sheet(sheet_name,df):
    # Load credentials from the .streamlit/secrets.toml file
    secrets = toml.load("/home/ciroshia/Downloads/github/comissao/.streamlit/secrets.toml")
    creds_info = secrets["connections"]["gsheets"]

    # Define scopes for accessing Google Sheets and Drive
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    # Create credentials and authorize the gspread client
    creds = Credentials.from_service_account_info(creds_info, scopes=scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet using the URL from the toml file
    spreadsheet = client.open_by_url(creds_info["spreadsheet"])

    # Select the first worksheet and fetch all records
    worksheet = spreadsheet.worksheet(sheet_name)
    worksheet.clear()
    set_with_dataframe(worksheet, df)

    return df

vmb_path = "assets/venda-mensal-bruta_colab-bi_2025-02-11.xlsx"
agendamento_path = "assets/agendamentos_colab-bi_2025-02-12 (1).xlsx"

venda_mensal_df = pd.read_excel(vmb_path)
agendamentos_df = pd.read_excel(agendamento_path)
procedimentos_df = get_sheetdata("procedimentos_padronizados")
prestadora_df = get_sheetdata("base_prestadoras")
comissao_df = get_sheetdata("comissoes")
tipo_prestadora_df = get_sheetdata("tipo_prestadora")
garantido_df = get_sheetdata("base_garantido")



In [7]:
# Todas as bases

base_compilada = criar_base_compilada(agendamentos_df,venda_mensal_df,procedimentos_df,prestadora_df,comissao_df,tipo_prestadora_df)
base_lavieen = criar_base_lavieen(base_compilada)
base_comissoes = criar_comissoes(base_compilada)
base_avaliacoes = cria_avaliacoes(base_compilada,"mensal")
revenda_df = cria_base_revenda(venda_mensal_df)
base_garantido = criar_base_garantido(base_compilada,garantido_df)
base_final = juntar_bases(base_comissoes,base_avaliacoes,revenda_df,base_lavieen,base_garantido)

/home/ciroshia/Downloads/github/comissao/auxiliar/arrumar_bases.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenda_df["Unidade"] = revenda_df["Unidade"].str.normalize("NFKC").str.strip().str.lower()
/home/ciroshia/Downloads/github/comissao/auxiliar/arrumar_bases.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenda_df["nome_prestadora"] = revenda_df["nome_prestadora"].str.normalize("NFKC").str.strip().str.lower()


# Rascunhos


In [11]:
base_final.columns

Index(['nome_prestadora', 'tipo_prestadora', 'Unidade', 'comissao_total',
       'avaliacoes_total', 'comissao_revenda', 'valor_lavieen',
       'comissao_garantido_total'],
      dtype='object')